# Skill estimation using graphical models

Let's see how we can use a simple model to predict a hidden "skill level" of players based on their performance in a collection of games against each other.  We need data (the games and outcomes), and a model of how skill translates into these outcomes (e.g., higher skilled players have a better chance of winning).

In [1]:
import sys
import os

sys.path.append(r'C:\Users\user\OneDrive - personalmicrosoftsoftware.uci.edu\2020Spring\CS179')
import pyGM as gm
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline         

### Processing data

In [11]:
def load_data(dir='data/', pKeep=1.0, nEdge=3, nKeep=5, opt='train'):
    with open(dir+opt+'.csv', encoding='utf-8') as f:
        lines = f.read().split('\n')

    p = 0
    playerid = {}
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10: 
            continue   # parse error or blank line
        player0,player1 = csv[1],csv[4]
        if player0 not in playerid:
            playerid[player0]=p
            p+=1
        if player1 not in playerid:
            playerid[player1]=p
            p+=1

    nplayers = len(playerid)
    playername = ['']*nplayers
    for player in playerid:
        playername[ playerid[player] ]=player  # id to name lookup


  # Sparsifying parameters (discard some training examples):
  # pKeep = 1.0   # fraction of edges to consider (immed. throw out 1-p edges)
  # nEdge = 3     # try to keep nEdge opponents per player (may be more; asymmetric)
  # nKeep = 5     # keep at most nKeep games per opponent pairs (play each other multiple times)
    
    games = []
    nplays, nwins = np.zeros( (nplayers,nplayers) ), np.zeros( (nplayers,nplayers) )
    for i in range(len(lines)):
        csv = lines[i].split(',')
        if len(csv) != 10:
            continue   # parse error or blank line
        a,b = playerid[csv[1]],playerid[csv[4]]
        aw,bw = csv[2]=='[winner]',csv[5]=='[winner]'
        if opt == 'valid':
            nplays[a,b] += 1
            nplays[b,a]+=1
            nwins[a,b] += aw
            nwins[b,a] += bw            
            continue
            
        if (np.random.rand() < pKeep):
            if (nplays[a,b] < nKeep) and ( ((nplays[a,:]>0).sum() < nEdge) or ((nplays[:,b]>0).sum() < nEdge) ):
                if a != b:
                    games.append((a, b, aw*2-1))
                nplays[a,b] += 1
                nplays[b,a]+=1
                nwins[a,b] += aw
                nwins[b,a] += bw
    
    return nplayers, nplays, nwins, games


In [3]:
nplayers, nplays, nwins, games = load_data()

In [4]:
print('summary: ', nplayers)
print(nplays.shape, nplays.sum())
print(nwins.shape, nwins.sum())
print('games', len(games))

summary:  999
(999, 999) 9354.0
(999, 999) 4696.0
games 4677


### A simple list of games & outcomes

In [5]:
# games = [
#     (0,2, +1),  # P0 played P2 & won
#     (0,2, +1),  # played again, same outcome
#     (1,2, -1),  # P1 played P2 & lost
#     (0,1, -1),  # P0 played P1 and lost
# ]

### Win probability and graphical model

In [6]:
# nplayers = max( [max(g[0],g[1]) for g in games] )+1
nlevels = 10   # let's say 10 discrete skill levels
scale = .3     # this scales how skill difference translates to win probability

# Make variables for each player; value = skill level
X = [None]*nplayers
for i in range(nplayers):
    X[i] = gm.Var(i, nlevels)   

# Information from each game: what does Pi winning over Pj tell us?
#    Win probability  Pr[win | Xi-Xj]  depends on skill difference of players
Pwin = np.zeros( (nlevels,nlevels) )
for i in range(nlevels):
    for j in range(nlevels):
        diff = i-j                   # find the advantage of Pi over Pj, then 
        Pwin[i,j] = (1./(1+np.exp(-scale*diff)))  # Pwin = logistic of advantage

# before any games, uniform belief over skill levels for each player:
factors = [ gm.Factor([X[i]],1./nlevels) for i in range(nplayers) ]

# Now add the information from each game:
for g in games:
    P1,P2,win = g[0],g[1],g[2]
    if P1>P2: P1,P2,win=P2,P1,-win  # (need to make player IDs sorted...)
    factors.append(gm.Factor([X[P1],X[P2]], Pwin if win>0 else 1-Pwin) )

In [7]:
model = gm.GraphModel(factors)
model.makeMinimal()  # merge any duplicate factors (e.g., repeated games)

In [8]:
if model.nvar < 0:       # for very small models, we can do brute force inference:
    jt = model.joint()
    jt /= jt.sum()       # normalize the distribution and marginalize the table
    bel = [jt.marginal([i]) for i in range(nplayers)] 
else:                    # otherwise we need to use some approximate inference:
    from pyGM.messagepass import LBP, NMF
    lnZ,bel = LBP(model, maxIter=10, verbose=True)   # loopy BP
    #lnZ,bel = NMF(model, maxIter=10, verbose=True)  # Mean field

Iter 1: 2328.845598665863
Iter 2: -2613.7165885218287
Iter 3: -1820.508077653653
Iter 4: -2317.6145273431475
Iter 5: -2308.1633503011126
Iter 6: -2323.273023919673
Iter 7: -2323.171184337802
Iter 8: -2324.046633286288
Iter 9: -2324.105782617845
Iter 10: -2324.1684010596146


The normalization constant, $\log(Z)$, represents the (log) probability of evidence for our model, namely the probability of the observed game outcomes given our parameters, etc.  We could experiment with changing the win probability function or its scaling parameter to try to make our model better fit the data using this value.

For example, if you play with "scale" on these toy data, you'll find scale=0 (so that every game is a 50-50 chance independent of skill level) fits the data pretty well, because the few outcomes listed are not really consistent with skill determining outcome.  But, if you change the data so that there is an obvious ordering of skill (e.g., P0 then 1 and/or 2), a larger scale parameter will better fit the data.

###  Ranking players by predicted skill

In [9]:
print("Mean skill estimates: ")
print([ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)] )

Mean skill estimates: 
[8.168431124827274, 8.106165643593911, 8.587439162046243, 8.996633845387574, 8.81433065165019, 5.720952960389763, 8.447029137555369, 5.422324544817899, 8.303425803518454, 8.629519617548185, 4.10788001317596, 6.393867282333231, 6.512061438467681, 2.980236316778229, 7.8901045950936055, 7.236562416754825, 2.4716157857849055, 2.0480383702343783, 2.7928059069633493, 8.12979358800112, 3.5307161297569785, 6.369195657330578, 8.029353931345607, 8.593336389940891, 8.97667018616674, 6.335216647309615, 8.999813494018957, 4.2469663446688966, 8.589138673254604, 7.000036729482235, 8.043533820300471, 2.9885399637371455, 4.956034635514705, 8.574358912440845, 4.53289206377241, 8.381403553428337, 3.0673450378119536, 5.094805317570846, 8.707551825537283, 7.670665550981718, 5.097073562643546, 8.250955361225486, 6.9396289862066, 7.846904824334132, 3.658431107174831, 7.69834862118398, 8.275700761682982, 8.442915703448776, 7.988356380657917, 6.898076098668252, 4.105975665957452, 5.25080

### Predicting match outcomes

In [12]:
i,j = np.random.randint(0, nplayers),  np.random.randint(0, nplayers)
print("Estimated probability P{} beats P{} next time:".format(i,j))
# Expected value (over skill of P0, P1) of Pr[win | P0-P1]
if i<j:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum() )
else:
    print( (bel[i]*bel[j]*gm.Factor([X[i],X[j]],1-Pwin)).table.sum() )
    
# Notes: we should probably use the joint belief over Xi and Xj, but for simplicity
#  with approximate inference we'll just use the estimated singleton marginals

Estimated probability P590 beats P174 next time:
0.5055734485654912


In [13]:
nplayers_val, nplays_val, nwins_val, games_val = load_data(opt = 'valid')

In [14]:
def logit(z): return 1./(1.+np.exp(-z))

def prediction_loss(skill, nplayers, nplays, nwins, games):
    
    loss = 0.
    binary_loss = 0.
    for i in range(nplayers):
        for j in range(i+1, nplayers):
            if nplays[i, j] == 0:
                continue
            prob = nwins[i,j] / nplays[i,j]
            prob_hat = (bel[i]*bel[j]*gm.Factor([X[i],X[j]],Pwin)).table.sum()
            loss += np.abs(prob_hat - prob)
            binary_loss += np.logical_xor(prob_hat >= 0.5, prob >= 0.5)
    
    loss /= (nplays > 0).sum()/2
    binary_loss /= (nplays > 0).sum()/2
    
    return loss, binary_loss


In [15]:
skill = [ bel[i].table.dot(np.arange(nlevels)) for i in range(nplayers)]

In [16]:
prediction_loss(skill, nplayers_val, nplays_val, nwins_val, games_val)

(0.4084186989651854, 0.4448548562769333)